In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from synth_dim_model import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import pandas as pd
import concurrent.futures
from matplotlib.animation import FuncAnimation

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def compute_values(mu_V_ratio, J_V_ratio, V, N, M, states, theta):
    """Compute energy gap and sigma for given parameters."""
    H = construct_rescaled_hamiltonian(N, M, V, mu_V_ratio, J_V_ratio, theta, boundary_conditions = "OBC")
    eigenvalues, eigenvectors = exact_diagonalize(H)
    energy_gap = eigenvalues[1] - eigenvalues[0]
    sigma = sigma_ij(0, 1, eigenvectors[0], states, N, M) / M
    return energy_gap, sigma

N = 2
M = 4

min_J_V_ratio = -5
max_J_V_ratio = 5
max_mu_V_ratio = 5
resolution = 100

theta = 0

mu_V_ratios = np.linspace(0.01, max_mu_V_ratio, resolution)
J_V_ratios = np.linspace(min_J_V_ratio, max_J_V_ratio, resolution)
mu_V_ratio_grid, J_V_ratio_grid = np.meshgrid(mu_V_ratios, J_V_ratios)


for V in [-1, 1]:
    sigma_grid = np.zeros_like(mu_V_ratio_grid)
    energy_gap_over_V_grid = np.zeros_like(mu_V_ratio_grid)
    states, _ = enumerate_states(N, M)
    
    # Use ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_index = {}
        
        for i, mu_V_ratio in enumerate(mu_V_ratios):
            for j, J_V_ratio in enumerate(J_V_ratios):
                future = executor.submit(compute_values, mu_V_ratio, J_V_ratio, V, N, M, states, theta)
                future_to_index[future] = (j, i)
        
        for future in concurrent.futures.as_completed(future_to_index):
            j, i = future_to_index[future]
            try:
                energy_gap, sigma = future.result()
                energy_gap_over_V_grid[j, i] = energy_gap
                sigma_grid[j, i] = sigma
            except Exception as e:
                print(f"Error computing values for indices ({j}, {i}): {e}")
    
    sign_V = "positive" if V > 0 else "negative"
    sign_str = r"$V > 0$" if sign_V == "positive" else r"$V < 0$"

    # Plot energy gap
    plt.figure(figsize=(6, 5))
    plt.pcolormesh(J_V_ratio_grid, mu_V_ratio_grid, np.log(1/energy_gap_over_V_grid), shading='auto', cmap='plasma')
    plt.colorbar(label="Energy Gap [log$(|V|/E)$]")
    plt.xlabel(r"$J/|V|$")
    plt.ylabel(r"$\mu/|V|$")
    plt.title(f"Energy Gap: $N={N}$, $M={M}$, {sign_str}, " + r"$\theta = $" + f"{np.round(theta,3)}")
    plt.tight_layout()
    plt.show()

    # Plot sigma
    plt.figure(figsize=(6, 5))
    plt.pcolormesh(J_V_ratio_grid, mu_V_ratio_grid, sigma_grid, shading='auto', cmap='plasma')
    plt.colorbar(label=r"$\sigma$")
    plt.xlabel(r"$J/V$")
    plt.ylabel(r"$\mu/V$")
    plt.title(f"$\sigma$: $N={N}$, $M={M}$, {sign_str}")
    plt.tight_layout()
    plt.show()


In [ ]:
plot_data(N,M, sign_V = "negative")
plot_data(N,M, sign_V = "positive")
plot_data(N,M, sign_V = "negative", gap_or_sigma = "sigma")
plot_data(N,M, sign_V = "positive", gap_or_sigma = "sigma")

In [ ]:
# Parameters
N = 2
M = 4
min_J_V_ratio = -5
max_J_V_ratio = 5
max_mu_V_ratio = 3
resolution = 100
V = -1  # can be changed if needed

mu_V_ratios = np.linspace(0.01, max_mu_V_ratio, resolution)
J_V_ratios = np.linspace(min_J_V_ratio, max_J_V_ratio, resolution)
mu_V_ratio_grid, J_V_ratio_grid = np.meshgrid(mu_V_ratios, J_V_ratios)
states, _ = enumerate_states(N, M)
thetas = np.linspace(0, np.pi, 100)

# Precompute data
energy_gap_data = []

for theta in thetas:
    sigma_grid = np.zeros_like(mu_V_ratio_grid)
    energy_gap_grid = np.zeros_like(mu_V_ratio_grid)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_index = {}
        for i, mu_V_ratio in enumerate(mu_V_ratios):
            for j, J_V_ratio in enumerate(J_V_ratios):
                future = executor.submit(compute_values, mu_V_ratio, J_V_ratio, V, N, M, states, theta)
                future_to_index[future] = (j, i)

        for future in concurrent.futures.as_completed(future_to_index):
            j, i = future_to_index[future]
            try:
                energy_gap, sigma = future.result()
                energy_gap_grid[j, i] = energy_gap
                sigma_grid[j, i] = sigma
            except Exception as e:
                print(f"Error computing values for indices ({j}, {i}): {e}")

    energy_gap_data.append(energy_gap_grid)

# Set up the plot
fig, ax = plt.subplots(figsize=(6, 5))
cmap = plt.get_cmap('plasma')
mesh = ax.pcolormesh(J_V_ratio_grid, mu_V_ratio_grid, np.log(1 / energy_gap_data[0]), shading='auto', cmap=cmap)
cbar = plt.colorbar(mesh, ax=ax, label="Energy Gap [log$(|V|/E)$]")
ax.set_xlabel(r"$J/|V|$")
ax.set_ylabel(r"$\mu/|V|$")
title = ax.set_title(f"$N={N}$, $M={M}$, $V > 0$, " + r"$\theta = $" + f"{np.round(thetas[0], 3)}")

# Animation function
def update(frame):
    data = energy_gap_data[frame]
    mesh.set_array(np.log(1 / data).ravel())
    title.set_text(f"$N={N}$, $M={M}$, $V > 0$, " + r"$\theta = $" + f"{np.round(thetas[frame], 3)}")
    return mesh, title

# Create animation
ani = FuncAnimation(fig, update, frames=len(thetas), blit=False, repeat=True)

# Show or save
plt.tight_layout()
plt.show()
ani.save("energy_gap_animation.gif", fps=25, dpi=150)
